A. Visning av COVID data

B. Pandemi modeller

C. COVID-API

#A. Visning av COVID data

## Steg 1



På nuværende tidspunkt findes adskillige kilder for corona virus.  

*   https://github.com/CSSEGISandData/COVID-19



For at kunne opnå en højere opløsning eller grandualieret kan det være nødvendigt hente lokal-specifik data.

Dette datasæt er hente ved af digitalisering af statiske billeder af grafer. 
http://getdata-graph-digitizer.com/

Denne metode tilføjere yderliger usikkerhed til validiteten af datasættet, men kan sometide lette dataindsamling.


In [1]:
#@title udvikling i corona smittet per fylke niveau
import pandas as pd
import altair as alt
url="https://raw.githubusercontent.com/jfrausig/test/master/covid_19_fylker_6.json"
data_json=pd.read_json(url)
alt.Chart(data_json).transform_fold(
    ['Oslo', 'Rogaland','Møre_og_Romsdal','Nordland','Viken','Innlandet','Vestfold_og_Telemark','Agder','Vestland','Trøndelag','Troms_og_Finnmark']
).mark_line().encode(
    x='Dag_nr:Q',
    y='value:Q',
    color='key:N'
).properties(
    width=800, height=800
).interactive()

alt.Chart(...)

## Steg 2

Data for Norge kan også hentes som API

In [2]:
#@title Data for Norge
import pandas as pd

data=pd.read_json("https://api.covid19api.com/total/country/norway")
data

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,Norway,,,,,0,0,0,0,0,0,2020-01-22 00:00:00+00:00
1,Norway,,,,,0,0,0,0,0,0,2020-01-23 00:00:00+00:00
2,Norway,,,,,0,0,0,0,0,0,2020-01-24 00:00:00+00:00
3,Norway,,,,,0,0,0,0,0,0,2020-01-25 00:00:00+00:00
4,Norway,,,,,0,0,0,0,0,0,2020-01-26 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
612,Norway,,,,,0,0,186554,850,0,185704,2021-09-25 00:00:00+00:00
613,Norway,,,,,0,0,186935,850,0,186085,2021-09-26 00:00:00+00:00
614,Norway,,,,,0,0,187593,850,0,186743,2021-09-27 00:00:00+00:00
615,Norway,,,,,0,0,188295,851,0,187444,2021-09-28 00:00:00+00:00


## Steg 3

Dette kan plottes såledelse

In [3]:
#@title Plot af udvikling af bekræftet korona virus inficeret
import altair as alt

source = data

line = alt.Chart(source).mark_line().encode(
    x='Date:T',
    y='Confirmed:Q'
)

band = alt.Chart(source).mark_errorband(extent='ci').encode(
    x='Date:T',
    y=alt.Y('Confirmed:Q', title='Confirmed'),
)

band + line

alt.LayerChart(...)

# B. Pandemi modeller

## Steg 1

SIR model

In [4]:
#@title SIR1.py - Programming for Computations - Python, kilde: p.110-124 Ipywidgets
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import zeros, linspace
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)

def update(S_init=50, I_init=1, R_init=1):

  # Time unit: 1 h
    beta = 10./(40*8*24)
    gamma = 3./(15*24)
    dt = 0.1             # 6 min
    D = 30               # Simulate for D days
    N_t = int(D*24/dt)   # Corresponding no of time steps

    t = linspace(0, N_t*dt, N_t+1)
    S = zeros(N_t+1)
    I = zeros(N_t+1)
    R = zeros(N_t+1)

    # Initial condition
    S[0] = S_init
    I[0] = I_init
    R[0] = R_init

    # Step equations forward in time
    for n in range(N_t):
        S[n+1] = S[n] - dt*beta*S[n]*I[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n]
        R[n+1] = R[n] + dt*gamma*I[n]
        
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1, 1, 1)
    #ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, S)
    ax.plot(t, I)
    ax.plot(t, R)

    fig.canvas.draw()

interact(update);

interactive(children=(IntSlider(value=50, description='S_init', max=150, min=-50), IntSlider(value=1, descript…

## Steg 2

SIR

In [ ]:
#@title SIR2.py - Programming for Computations - Python, kilde: p.110-124


from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import zeros, linspace
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)

def update(S_init=50, I_init=1, R_init=0):


    # Time unit: 1 h
    #beta = 10./(40*8*24)
    #beta = 0.00033            # ca. beta/4, i.e. reduced compared to SIR1.py
    beta = 0.00043
    gamma = 3./(15*24)
    dt = 0.1             # 6 min
    D = 300              # Simulate for D days
    N_t = int(D*24/dt)   # Corresponding no of hours
    #nu = 1./(24*50)      # Average loss of immunity: 50 days
    nu = 1./(24*90)      # Average loss of immunity: 90 days

    t = np.linspace(0, N_t*dt, N_t+1)
    S = np.zeros(N_t+1)
    I = np.zeros(N_t+1)
    R = np.zeros(N_t+1)

    # Initial condition
    S[0] = S_init
    I[0] = I_init
    R[0] = R_init

    # Step equations forward in time
    for n in range(N_t):
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*nu*R[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*nu*R[n]
        
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1, 1, 1)
    #ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, S)
    ax.plot(t, I)
    ax.plot(t, R)

    fig.canvas.draw()

interact(update);

interactive(children=(IntSlider(value=50, description='S_init', max=150, min=-50), IntSlider(value=1, descript…

## Steg 3

# SIRV model

In [ ]:
#@title SIR1V.py - Programming for Computations - Python, kilde: p.110-124


from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import zeros, linspace
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)

def update(S_init=50, I_init=1, R_init=0,V_init=0):

    # Time unit: 1 h
    beta = 10./(40*8*24)
    beta = beta=0.00033  # reduce beta to ca. beta/4 compared to SIR1.py
    #print('beta:', beta)
    gamma = 3./(15*24)
    dt = 0.1             # 6 min
    D = 300              # Simulate for D days
    N_t = int(D*24/dt)   # Corresponding no of hours
    nu = 1./(24*50)      # Average loss of immunity: 50 days
    #p = 0.0005
    p = 0.0001

    t = np.linspace(0, N_t*dt, N_t+1)
    S = np.zeros(N_t+1)
    I = np.zeros(N_t+1)
    R = np.zeros(N_t+1)
    V = np.zeros(N_t+1)

    # Initial condition
    S[0] = S_init
    I[0] = I_init
    R[0] = R_init
    V[0] = V_init

    # Step equations forward in time
    for n in range(N_t):
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*nu*R[n] - dt*p*S[n]
        V[n+1] = V[n] + dt*p*S[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*nu*R[n]
        loss = int(V[n+1] + S[n+1] + R[n+1] + I[n+1]) - \
              int(V[0] + S[0] + R[0] + I[0])
        #if loss > 0:
          #  print('loss: {:d}'.format(loss))
        
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1, 1, 1)
    #ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, S)
    ax.plot(t, I)
    ax.plot(t, R)

    fig.canvas.draw()

interact(update);

interactive(children=(IntSlider(value=50, description='S_init', max=150, min=-50), IntSlider(value=1, descript…

## Steg 4

# SIRV model

In [ ]:
#@title SIR2.py - Programming for Computations - Python, kilde: p.110-124


from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import zeros, linspace
import matplotlib.pyplot as plt

x = np.linspace(0, 2 * np.pi)

def update(S_init=50, I_init=1, R_init=0,V_init=0):

    beta = 10./(40*8*24)
    beta /= 4            # Reduce beta compared to SIR1.py
    #print('beta:', beta)
    gamma = 3./(15*24)
    dt = 0.1             # 6 min
    D = 100              # Simulate for D days
    N_t = int(D*24/dt)   # Corresponding no of hours
    nu = 1./(24*50)      # Average loss of immunity: 50 days

    t = np.linspace(0, N_t*dt, N_t+1)
    S = np.zeros(N_t+1)
    I = np.zeros(N_t+1)
    R = np.zeros(N_t+1)
    V = np.zeros(N_t+1)

    # Vaccination campaign
    p = np.zeros(N_t+1)
    start_index = int(6*24/dt)  # 6 days = 6*24 h, div. by dt=0.1 gives intervals
    stop_index = int(15*24/dt)
    p[start_index:stop_index] = 0.005

    # Initial condition
    S[0] = S_init
    I[0] = I_init
    R[0] = R_init
    V[0] = V_init

    # Step equations forward in time
    for n in range(N_t):
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*nu*R[n] - dt*p[n]*S[n]
        V[n+1] = V[n] + dt*p[n]*S[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*nu*R[n]
        loss = int(V[n+1] + S[n+1] + R[n+1] + I[n+1]) - \
              int(V[0] + S[0] + R[0] + I[0])
        
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(1, 1, 1)
    #ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, S)
    ax.plot(t, I)
    ax.plot(t, R)
    ax.plot(t, V)

    fig.canvas.draw()

interact(update);

interactive(children=(IntSlider(value=50, description='S_init', max=150, min=-50), IntSlider(value=1, descript…

# C. COVID-API

## Steg 1

#Henting af koronavirus data

For en sikkerhedsskyld kan følgende bibliotekker installeres

In [5]:
#@title installation af bibliotekker
!pip install httplib2
!pip install urllib3

## Steg 2

Nationale tilfælde af inficeret kan hentes

In [6]:
#@title Nationale tilfælde
import pandas as pd
########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '0dedd117909940a19e1d76eab2cd829c',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('api.helsedirektoratet.no')
    conn.request("GET", "/ProduktCovid19/Covid19Statistikk/nasjonalt?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

b'{"registreringer":[{"dato":"2020-03-08T00:00:00","antInnlagte":0,"antRespirator":0},{"dato":"2020-03-09T00:00:00","antInnlagte":2,"antRespirator":0},{"dato":"2020-03-10T00:00:00","antInnlagte":8,"antRespirator":0},{"dato":"2020-03-11T00:00:00","antInnlagte":13,"antRespirator":0},{"dato":"2020-03-12T00:00:00","antInnlagte":23,"antRespirator":1},{"dato":"2020-03-13T00:00:00","antInnlagte":27,"antRespirator":4},{"dato":"2020-03-14T00:00:00","antInnlagte":35,"antRespirator":4},{"dato":"2020-03-15T00:00:00","antInnlagte":41,"antRespirator":6},{"dato":"2020-03-16T00:00:00","antInnlagte":50,"antRespirator":10},{"dato":"2020-03-17T00:00:00","antInnlagte":68,"antRespirator":13},{"dato":"2020-03-18T00:00:00","antInnlagte":82,"antRespirator":16},{"dato":"2020-03-19T00:00:00","antInnlagte":102,"antRespirator":19},{"dato":"2020-03-20T00:00:00","antInnlagte":130,"antRespirator":26},{"dato":"2020-03-21T00:00:00","antInnlagte":148,"antRespirator":28},{"dato":"2020-03-22T00:00:00","antInnlagte":169,"

## Steg 3

Datasættet kan printes såledelse

In [7]:
#@title print af datasæt
pd.DataFrame.from_dict(pd.read_json(data)['registreringer'])

,registreringer
0,"{'dato': '2020-03-08T00:00:00', 'antInnlagte':..."
1,"{'dato': '2020-03-09T00:00:00', 'antInnlagte':..."
2,"{'dato': '2020-03-10T00:00:00', 'antInnlagte':..."
3,"{'dato': '2020-03-11T00:00:00', 'antInnlagte':..."
4,"{'dato': '2020-03-12T00:00:00', 'antInnlagte':..."
...,...
491,"{'dato': '2021-09-24T00:00:00', 'antInnlagte':..."
492,"{'dato': '2021-09-27T00:00:00', 'antInnlagte':..."
493,"{'dato': '2021-09-28T00:00:00', 'antInnlagte':..."
494,"{'dato': '2021-09-29T00:00:00', 'antInnlagte':..."


## Steg 4

Data fra hver helseregion kan også vises

In [8]:
#@title Per helseregion
########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '0dedd117909940a19e1d76eab2cd829c',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('api.helsedirektoratet.no')
    conn.request("GET", "/ProduktCovid19/Covid19Statistikk/helseregion?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

b'[{"id":"0007-0042-Midt","navn":"Midt","registreringer":[{"dato":"2020-03-08T00:00:00","antInnlagte":0,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-09T00:00:00","antInnlagte":0,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-10T00:00:00","antInnlagte":1,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-11T00:00:00","antInnlagte":1,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-12T00:00:00","antInnlagte":2,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-13T00:00:00","antInnlagte":2,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-14T00:00:00","antInnlagte":3,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-15T00:00:00","antInnlagte":3,"antRespirator":0,"antCovidIntensiv":0},{"dato":"2020-03-16T00:00:00","antInnlagte":3,"antRespirator":1,"antCovidIntensiv":0},{"dato":"2020-03-17T00:00:00","antInnlagte":3,"antRespirator":1,"antCovidIntensiv":0},{"dato":"2020-03-18T00:00:00","antInnlagte":6,"antRespirator":2,"antCovidIntensiv"

## Steg 5

Print af tilfælde per helseregion kan printes såledelse

In [9]:
#@title print for en specifik helseregion "0"
pd.read_json(data)['registreringer'][0]

[{'antCovidIntensiv': 0,
  'antInnlagte': 0,
  'antRespirator': 0,
  'dato': '2020-03-08T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 0,
  'antRespirator': 0,
  'dato': '2020-03-09T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 1,
  'antRespirator': 0,
  'dato': '2020-03-10T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 1,
  'antRespirator': 0,
  'dato': '2020-03-11T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 2,
  'antRespirator': 0,
  'dato': '2020-03-12T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 2,
  'antRespirator': 0,
  'dato': '2020-03-13T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 3,
  'antRespirator': 0,
  'dato': '2020-03-14T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 3,
  'antRespirator': 0,
  'dato': '2020-03-15T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 3,
  'antRespirator': 1,
  'dato': '2020-03-16T00:00:00'},
 {'antCovidIntensiv': 0,
  'antInnlagte': 3,
  'antRespirator': 1,
  'dato': '2020-03-17T00

Steg 6

Data for hvert helseforetak kan også hentes

In [ ]:
#@title Data for Helseforetak

####################################

########### Python 3.2 #############
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': '0dedd117909940a19e1d76eab2cd829c',
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('api.helsedirektoratet.no')
    conn.request("GET", "/ProduktCovid19/Covid19statistikk/helseforetak?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

####################################

b'[{"id":"0007-0042-Akershus-universitetssykehus-HF","helseforetakNavn":"Akershus universitetssykehus HF","region":"S\xc3\xb8r-\xc3\x98st","covidRegistreringer":[{"dato":"2020-03-08T00:00:00","antInnlagte":0},{"dato":"2020-03-09T00:00:00","antInnlagte":0},{"dato":"2020-03-10T00:00:00","antInnlagte":2},{"dato":"2020-03-11T00:00:00","antInnlagte":2},{"dato":"2020-03-12T00:00:00","antInnlagte":3},{"dato":"2020-03-13T00:00:00","antInnlagte":3},{"dato":"2020-03-14T00:00:00","antInnlagte":5},{"dato":"2020-03-15T00:00:00","antInnlagte":7},{"dato":"2020-03-16T00:00:00","antInnlagte":8},{"dato":"2020-03-17T00:00:00","antInnlagte":10},{"dato":"2020-03-18T00:00:00","antInnlagte":9},{"dato":"2020-03-19T00:00:00","antInnlagte":10},{"dato":"2020-03-20T00:00:00","antInnlagte":18},{"dato":"2020-03-21T00:00:00","antInnlagte":20},{"dato":"2020-03-22T00:00:00","antInnlagte":23},{"dato":"2020-03-23T00:00:00","antInnlagte":31},{"dato":"2020-03-24T00:00:00","antInnlagte":30},{"dato":"2020-03-25T00:00:00","a

## Steg 7

Ligeledelse kan registering per helseforetak vises

In [ ]:
#@title print for helseforetak
pd.read_json(data)['covidRegistreringer'][0]

[{'antInnlagte': 0, 'dato': '2020-03-08T00:00:00'},
 {'antInnlagte': 0, 'dato': '2020-03-09T00:00:00'},
 {'antInnlagte': 2, 'dato': '2020-03-10T00:00:00'},
 {'antInnlagte': 2, 'dato': '2020-03-11T00:00:00'},
 {'antInnlagte': 3, 'dato': '2020-03-12T00:00:00'},
 {'antInnlagte': 3, 'dato': '2020-03-13T00:00:00'},
 {'antInnlagte': 5, 'dato': '2020-03-14T00:00:00'},
 {'antInnlagte': 7, 'dato': '2020-03-15T00:00:00'},
 {'antInnlagte': 8, 'dato': '2020-03-16T00:00:00'},
 {'antInnlagte': 10, 'dato': '2020-03-17T00:00:00'},
 {'antInnlagte': 9, 'dato': '2020-03-18T00:00:00'},
 {'antInnlagte': 10, 'dato': '2020-03-19T00:00:00'},
 {'antInnlagte': 18, 'dato': '2020-03-20T00:00:00'},
 {'antInnlagte': 20, 'dato': '2020-03-21T00:00:00'},
 {'antInnlagte': 23, 'dato': '2020-03-22T00:00:00'},
 {'antInnlagte': 31, 'dato': '2020-03-23T00:00:00'},
 {'antInnlagte': 30, 'dato': '2020-03-24T00:00:00'},
 {'antInnlagte': 42, 'dato': '2020-03-25T00:00:00'},
 {'antInnlagte': 49, 'dato': '2020-03-26T00:00:00'},
 {'